# Bag-of-Words for Document Classification

## Data and problem statement

We consider a situation where a list of English language documents are given and a class to which each of these belongs. We would like to extract a set of features from each document (in a consistent manner) and to choose a classification model to train on these data. As always, we need to split our data into train, validation and test partition or apply k-fold cross validation. In this training we will simplify by simply splitting this documents list at random into train and test parts (0.70 train and 0.30 test).

> For this training you'll need the *uci-news-aggregator.csv* dataset from [this kaggle project](https://www.kaggle.com/uciml/news-aggregator-dataset). You'll need to download it if you don't have it already.

Now we will limit our document dataset to two classes (b = business, e = entertainment) so that we have a binary classification problem:

## Basic text preprocessing with NLTK

Since language is simply too rich to define separate variable for each word form that appears in or documents, we first transform our texts as follows, and thereby reduce number of distinct words.

1. Remove all punctuation signs and digits
2. Cast all letters to lower case
3. Remove words that appear very often (thus are little informative); we use a file from a package where standard list of stop words is found
4. Apply a lematization algorithm; there are many variants and essentially we wish to map e.g. ‘be’, ‘being’, ‘am’ , ‘is’ to a single word ‘be’; likewise ‘element’, ‘elements’, ‘elementary’, ‘elemental’ should be mapped to ‘element’. However ‘news’ should not map to ‘new’ as it is a distinct word. There are many scientific articles about how to do this in English language, and off course less for other languages and the proposed solutions also vary. 

## Creating binary features from documents

A human typed text is a rich representation of information which essentially requires human level of intelligence to fully understand. In particular order of words, punctuation signs, and exact grammatical forms in which words are used gives many different flavours to a piece of text. However we are only interested to separate a list of documents in two piles-one of class 0 and the other of class 1.

**Simplest approach one can take is to define one binary variable for each word that appears in any of the documents that we have to work with in this problem, in such way that if that word appears in a given document, then this feature has value 1 and of it does not appear then it has value 0.**

In this approach, if a word appears more than once, then the corresponding feature still has value 1. Also, the order of the words does not influence the extracted feature values, and also punctuation signs are disregarded all together (in fact we will remove them at the beginning of our processing).

As example ‘Story was telling about a data scientist’ and ‘Data scientist was telling a story about data’ will have exactly the same feature values sequence. As inadequate as this may seem, if the goal is to find documents about data scientists then it might in fact work well enough.

## Naive Bayes Classifier with binary features

One of the most simple types of machine learning models that is suitable here is the so called Naïve Bayes model. The simplest form of this model is of the binary classification type. There we assume that each observation consists of a vector of binary valued features $X=(x_1,x_2,…,x_k)$ (each component $x_j$ is either 0 or 1 valued) and a binary target value $Y$. By the basic Bayes theorem we have that

\begin{align*} P(Y \mid x_1,x_2,...,x_k) &= \frac{P(Y)\ P(x_1,x_2,...,x_k \mid Y)}{P(x_1,x_2,...,x_k)} \\
&= \frac{P(Y)\ P(x_1,x_2,...,x_k \mid Y)}{P(x_1,x_2,...,x_k\mid Y = 0)\ P(Y=0) + P(x_1,x_2,...,x_k\mid Y = 1)\ P(Y=1)}\end{align*}

Here $X=(x_1,x_2,…,x_k)$ is a binary valued sequence of feature values for a given observation. In this case we 'only' need to estimate probabilities $P(Y=1)$, $P(Y=0)$, and $P(x_1,x_2,…,x_k|Y=0)$, $P(x_1,x_2,…,x_k|Y=1)$ for each possible binary sequence $x_1,x_2,...,x_k$ of feature values. Now for a feature vector of length $k$, there are $2^k$ distinct binary sequences of length $k$ (in each position we can have 0 or 1, so we indeed have to take k-th power of 2). So total number of parameters to estimate is $2\cdot2^k +2$ = 'waaay too many!'.

To get a feeling of this number, if we have many features, say as many as there are different words in a list of documents, of we would work with 200 different words, then $2^{200}=16^{50}$ which is a number larger then number of atoms in the universe! Also, Some of these combinations of binary values are typically very rare in our data which amounts to very low significance (certainty) of our estimate, and that translates directly to low predictive performance of our model.

## Reducing number of parameters

A way to mitigate a too high number of parameters to estimate is the so called Naïve Bayes assumption.  We thus assume that

$$ P(x_1,x_2,...,x_k \mid Y=0) = \prod_{i=1}^{k}P(x_i \mid Y=0)$$

and

$$ P(x_1,x_2,...,x_k \mid Y=1) = \prod_{i=1}^{k}P(x_i \mid Y=1)$$

for each sequence of binary values $x_1, x_2, …, x_k$.

This greatly reduces the number of probabilities we need to estimate, now only $k$ for $p(x_1,x_2,...,x_k)$. This means a total of $2\cdot k + 2$, for a total speedup of $\frac{2\cdot k + 2}{2 \cdot 2^k + 2}$. This speedup as a function of $k$ is plotted below:

We can estimate these probabilities using a maximum likelihood estimator, which gives us the following estimators:

$$p^i_{1,0} = P(x_i=1\mid Y=0) := \frac{\#\{\text{observations with $x_i = 1$ and $y = 0$}\} + 1}{\#\{\text{observations with $Y=0$}\} + 2}$$

$$p^i_{0,0} = P(x_i=0\mid Y=0) := \frac{\#\{\text{observations with $x_i = 0$ and $y = 0$}\} + 1}{\#\{\text{observations with $Y=0$}\} + 2}$$

$$ p^i_{0,1} = P(x_i=0\mid Y=1) := \frac{\#\{\text{observations with $x_i = 0$ and $y = 1$}\} + 1}{\#\{\text{observations with $Y=1$}\} + 2}.$$

$$ p^i_{1,1} = P(x_i=1\mid Y=1) := \frac{\#\{\text{observations with $x_i = 1$ and $y = 1$}\} + 1}{\#\{\text{observations with $Y=1$}\} + 2}.$$

and

$$ q^0 = P(Y=0) := \frac{\#\{\text{observations with $Y = 0$}\} + 1}{\#\{\text{total observations}\} + 2}.$$

$$ q^1 = P(Y=1) := \frac{\#\{\text{observations with $Y = 1$}\} + 1}{\#\{\text{total observations}\} + 2}.$$

Since this is binary classification, we could also calculate some of these probabilites by using $p^i_{0,0} = 1 - p^i_{1,0}$. We show the complete calculations above as this more easily generalizes to non-binary classification.

> *Note that we add 1 to the numerator and 2 to the denominator. This is what's called Laplace smoothing, which improves the estimator's accuracy and makes it more robust. You can read more about it [here](https://www.wikiwand.com/en/Additive_smoothing).*


Although above we show how each individual $p^i_{1,0}$ is calculated, it is easier to calculate the vector 

$$p_{1,0} = \big(p^1_{1,0}, p^2_{1,0}, ..., p^k_{1,0}\big)$$

where $p^i_{1,0} = P(x_i = 1 \mid Y=0)$. The same counts for the other probabilities $p^i_{0,1}, q^0,$ etc...

## How to classify a new documents

Say we have a new out-of-sample title we want to classify. The feature vector associated with sentence (after all the necessary cleaning steps) is $X = (x_1,x_2,...,x_k)$, where $x_i = 1$ if the $i$-th word is in the new sentence, and $x_i = 0$ if not. We can then calculate the new probability using:

$$ P(Y = 1 \mid x_1,...,x_k) = \frac{q^1\cdot \left(\prod_{i=1}^{k}p^i_{x_i,1}\right)}{q^0\cdot \left(\prod_{i=1}^{k}p^i_{x_i,0}\right) + q^1\cdot \left(\prod_{i=1}^{k}p^i_{x_i,1}\right)}$$

Now to calculate this probability, we will use an example sentence valued `title = "President trump actually launched a nuclear missile at the moon. We're doomed!"`. We will use the shorter vector notation $p_{0,0}$ and $q^0$ mentioned earlier, as this works nicely with numpy arrays.

We can see that our text was properly preprocessed, and we now have our sentence's feature vector. Lets get to calculated it's probability. We'll first calculate the $q^1\cdot \left(\prod_{i=1}^{k}p^i_{x_i,1}\right)$ term:

As you can see, the individual probabilities are quite small. For larger dictionaries Luckily some creative minds came up with as solution to this problem. Have a look at [this stackexchange post](https://stats.stackexchange.com/questions/105602/example-of-how-the-log-sum-exp-trick-works-in-naive-bayes) for a great explanation. We instead calculate the following, which avoids the issues of numerical underflow using the magic of logarithms:

$$\log\big(P(Y=1\mid x_1,...,x_k)\big) = \log\left(q^1\right) + \sum_{i=1}^k\log\left(p_{x_i,1}^i\right) - M$$

with

$$ M = A + \log\left(e^{\log\left(q_0\right) + \sum_{i=1}^k\log\left(p^i_{x_i,0}\right) - A} + e^{\log\left(q_1\right) + \sum_{i=1}^k\log\left(p^i_{x_i,1}\right) - A}\right)$$

where we take $A:= \max\left(\log\left(q_0\right) + \sum_{i=1}^k\log\left(p^i_{x_i,0}\right), \log\left(q_1\right) + \sum_{i=1}^k\log\left(p^i_{x_i,1}\right)\right)$. Let's try this out and see if we are able to avoid the underflow issue we had before.

Great, so we can see that it returns the same probabilities as when we calculated without the log-sum-exp trick. Now lets combine all the work we've done so far into a Naive Bayes Classifier class, which we can use in the future.